In [12]:
#--> IMPORTS

# Utilities
import numpy as np
import csv
import cv2
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Keras models
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3


# Keras utils
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, ELU
from keras.models import Model, Sequential

In [2]:
#--> CONSTANTS
DATA_DIR = 'data/'
DATA_FILE = 'driving_log.csv'

BATCH_SIZE = 128
EPOCHS = 2

In [3]:
#--> DATA PREPARATION
# Function for uploading data from file    
def load_file():
    # Load CSV file
    lines = []
    with open(DATA_DIR+'/'+DATA_FILE) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)  # skip the headers
        for line in reader:
            lines.append(line)

    # Process data
    images = []
    steerings = []
    for line in lines:
        source_path = DATA_DIR + line[0]
        image = cv2.imread(source_path)
        images.append(image)
        angle = float(line[3])
        steerings.append(angle)
    images = np.array(images)
    steerings = np.array(steerings)
    
    return images, steerings
    
# Upload from pickle (this saves processing time) or file
# Pickle file
data_file = 'temp/data_file.p'
try:
    with open(data_file, mode='rb') as f:
        data = pickle.load(f)
    images, steerings = data['images'], data['steerings']
except OSError as err:
    images, steerings = load_file()
    data = {'images':images, 'steerings':steerings}
    pickle.dump( data, open( data_file, "wb" ) )
X_train, X_valid, y_train, y_valid = train_test_split(images, steerings, test_size = 0.2)

#### Image augmentation techniques to test:
- Flip horizontally --> inverse steering angle
- Shifth horizontally --> adjust steering angle
- Use left and right cameras --> adjust steering angle
- Brightness, salt&pepper, shadow


In [4]:
# Check upload
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

(6428, 160, 320, 3) (6428,) (1608, 160, 320, 3) (1608,)


In [5]:
# have a look at keras ImageDataGenerator https://keras.io/preprocessing/image/
# Training data generator
def data_generator(images, steerings, batch_size = BATCH_SIZE):
    batch_train = np.zeros((batch_size, 160, 320, 3), dtype = np.float32)
    batch_angle = np.zeros((batch_size,), dtype = np.float32)
    while True:
        data, angle = shuffle(images, steerings)
        for i in range(batch_size):
          choice = int(np.random.choice(len(data),1))
          batch_train[i] = data[choice]
          batch_angle[i] = angle[choice]

        yield batch_train, batch_angle

train_data_generator = data_generator(X_train, y_train, 16)
valid_data_generator = data_generator(X_valid, y_valid, 16)

In [6]:
# Check generator
counter = 0
for features, values in train_data_generator:
    print('Train', features.shape)
    print('Train', values.shape)
    if counter > -1: break
    counter += 1
counter = 0
for features, values in valid_data_generator:
    print('Valid', features.shape)
    print('Valid', values.shape)
    if counter > -1: break
    counter += 1

Train (16, 160, 320, 3)
Train (16,)
Valid (16, 160, 320, 3)
Valid (16,)


In [13]:
# Create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(160, 320, 3))

# Add new dense layer on top
# Get the output of the model
x = base_model.output
# Flatten it
x = Flatten(name='flatten')(x)
# Add a fully-connected layer with a relu activation function
x = Dense(8, activation='relu')(x)
# Finally a dense with a single regression 1 output
predictions = Dense(1, W_regularizer = l2(0.001))(x)

# Model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Train only the top layers (which were randomly initialized)
# So, freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr = 0.0001), loss='mse', metrics=['accuracy'])
model.summary()

if 1 == 1:
    # train the model on the new data for a few epochs
    model.fit_generator(train_data_generator,
                        samples_per_epoch = len(X_train), nb_epoch = EPOCHS,
                        validation_data = valid_data_generator, validation_steps = 10)#, nb_val_samples = len(X_valid))

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of bcbd6486424b2319ff4ef7d526e38f63 so we will re-download the data.
86548480/87910968 [============================>.] - ETA: 0s

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
conv2d_9 (Conv2D)                (None, 79, 159, 32)   864         input_4[0][0]                    
____________________________________________________________________________________________________
batch_normalization_9 (BatchNorm (None, 79, 159, 32)   96          conv2d_9[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 79, 159, 32)   0           batch_normalization_9[0][0]      
___________________________________________________________________________________________

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=6428, validation_steps=10, validation_data=<generator..., epochs=2)`


KeyboardInterrupt: 

In [ ]:
# Very simple trainer to test
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape = (160, 320, 3)))
model.add(Flatten(input_shape= (160, 320, 3)))
model.add(Dense(1))

# Train simple model
model.compile(loss = 'mse', optimizer = 'adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 2)